In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=36686 sha256=26b457af23189e7c3fcecf943cf2543d4a769c7d9bc7db57975ee19acc248932
  Stored in directory: /tmp/pip-ephem-wheel-cache-v2r9ppq6/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import log_reg_xsubject_test



In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/log_reg/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for RNN training
epochs = 40
batch_size = 2


nreps = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']


for src_subject_id in range(1,nsubjects+1):

    results_df = log_reg_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            epochs = epochs, batch_size = batch_size, mv = 11, permute = False)

    results_df['Shuffled'] = False

    perm_results_df = log_reg_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            epochs = epochs, batch_size = batch_size, mv = 11, permute = True)

    perm_results_df['Shuffled'] = True

    #concatenate

    results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
    results_df['Train_Subject'] = src_subject_id

    # #save results to file
    results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

Streaming output truncated to the last 5000 lines.
(1223, 16)
(395,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/22
Evaluate Model
(1205, 16)
(383,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/23
Evaluate Model
(1167, 16)
(444,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/24
Evaluate Model
(1182, 16)
(439,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/25
Evaluate Model
(1181, 16)
(394,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/26
Evaluate Model
(1090, 16)
(404,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/27
Evaluate Model
(1076, 16)
(422,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/28
Evaluate Model
(991, 16)
(332,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/29
Evaluate Model
(1068, 16)
(419,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/30
Evaluate Model
(1501, 16)
(438,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_dat

Target Subject :/content/drive/MyDrive/limb-position-EMG-Repo/EMG_data/02
Evaluate Model


In [46]:
results_df.groupby(['Type','Shuffled']).mean()

Rep  Test_Subject  f1_score  accuracy_score  Train_Subject
Type  Shuffled                                                            
Test  False     1.5           2.5  0.572786        0.611005            1.0
      True      1.5           2.5  0.219632        0.269438            1.0
Train False     2.0           1.0  1.000000        1.000000            1.0
      True      2.0           1.0  0.495966        0.526869            1.0